<a href="https://colab.research.google.com/github/kharyal/machine-learning/blob/master/python/MNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt


In [6]:
batch_size=90
train = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())
train_batch = torch.utils.data.DataLoader(dataset = train,
                                             batch_size = batch_size,
                                             shuffle = True)

test_batch = torch.utils.data.DataLoader(dataset = test,
                                      batch_size = batch_size, 
                                      shuffle = False)


  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 23197401.58it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 422210.95it/s]
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 5492150.82it/s]                           
8192it [00:00, 183333.09it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


neural network

In [15]:
print(train.data.shape)   #### it is 60000 images with 28 by 28 dimension
# print(train.data[0].view(-1,28*28))

torch.Size([60000, 28, 28])


In [0]:
class net(nn.Module):
  def __init__(self):
    super(net,self).__init__()
    self.conv1=nn.Conv2d(1,32,3)
    self.conv2=nn.Conv2d(32,64,3)
#     self.pool1=nn.MaxPool2d(2,2)
    self.fc1=nn.Linear(36864,1000)
    self.fc2=nn.Linear(1000,100)
    self.fc3=nn.Linear(100,10)
  
  def forward(self,x):
    x=F.relu(self.conv1(x))
    x=F.relu(self.conv2(x))
#     x=self.pool1(x)
    x=x.view(-1,x.size(1) * x.size(2) * x.size(3))
    x=self.fc1(x)
    x=self.fc2(x)
    x=self.fc3(x)
    return x

In [0]:
network=net()
if torch.cuda.is_available():
  network.cuda()
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(network.parameters(), lr=0.0001)

In [96]:
for epoch in range(15):
  lossTotal=0
  for i, (images, labels) in enumerate(train_batch):
    images=Variable(images).cuda()
    labels=Variable(labels).cuda()
    
    optimizer.zero_grad()
    op=network(images)
    loss=loss_function(op,labels)
    loss.backward()
    lossTotal=lossTotal+loss.item()
    optimizer.step()
    if i%100 ==0:
      print("local loss: ",loss.item())
  print("######## end of EPOCH: ",epoch+1," LOSS: ",lossTotal," ########")

local loss:  2.2981302738189697
local loss:  0.3304274380207062
local loss:  0.24411562085151672
local loss:  0.273377388715744
local loss:  0.15849734842777252
local loss:  0.02946898154914379
local loss:  0.15007007122039795
######## end of EPOCH:  1  LOSS:  180.63631086423993  ########
local loss:  0.03627416864037514
local loss:  0.019713614135980606
local loss:  0.03589313477277756
local loss:  0.10406375676393509
local loss:  0.016749700531363487
local loss:  0.014924049377441406
local loss:  0.11905892938375473
######## end of EPOCH:  2  LOSS:  49.78826926834881  ########
local loss:  0.06346301734447479
local loss:  0.014422337524592876
local loss:  0.05055011063814163
local loss:  0.06800217926502228
local loss:  0.03870650380849838
local loss:  0.01603587530553341
local loss:  0.00729379104450345
######## end of EPOCH:  3  LOSS:  33.63399292132817  ########
local loss:  0.12592044472694397
local loss:  0.008610174059867859
local loss:  0.10284596681594849
local loss:  0.01267

In [102]:
correct=0
total=0
for images,labels in test_batch:
  images=Variable(images).cuda()
  labels=Variable(labels).cuda()
  
  output = network(images)
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  total += labels.size(0)
print("accuracy: ",(100*correct)/(total+1))

accuracy:  tensor(98, device='cuda:0')
